In [25]:
# Install dependencies
# Based on the libraries used in the project (geopandas, pandas, etc.)
!pip install geopandas pandas beautifulsoup4 shapely folium mapclassify matplotlib rtree pyproj h3pandas --quiet

In [26]:
!pip install duckdb -q

In [27]:
!pip install leafmap -q

In [28]:
import geopandas as gpd
import h3pandas
from shapely.geometry import Point, Polygon
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [29]:
import duckdb

# 1. SETUP: Connect to a persistent database file
#    This creates 'sport_equity.db' so you don't have to reload data every time.
db_path = f"sport_equity.db"
con = duckdb.connect(db_path)

# 2. EXTENSIONS: Install Spatial support (Crucial for your maps)
print("Installing Spatial Extension...")
con.install_extension("spatial")
con.load_extension("spatial")

Installing Spatial Extension...


In [30]:
import os

raw_path = os.path.abspath("../data/data_raw")
raw_path

'/home/amber/Equity_Analysis/data/data_raw'

In [31]:
# ==========================================
# PART A: BATCH IMPORT DEMOGRAPHICS (CSVs)
# ==========================================
print("Importing Top Sports & Exercise CSVs...")
# This reads all "Top Sports" files at once and adds a 'source_file' column
# so you know which row came from "Malays", "Females", etc.
# Using a broader wildcard 'Top Sports & Exercise*.csv' to avoid issues with the specific dash character
# And union_by_name=True to handle schema mismatches
con.sql(f"""
    CREATE OR REPLACE TABLE demographics_sports AS 
    SELECT * 
    FROM read_csv_auto('{raw_path}/Top Sports & Exercise*.csv', filename=true, union_by_name=true);
""")

# Import Barriers and Motivations
print("Importing Barriers & Motivations...")
con.sql(f"""
    CREATE OR REPLACE TABLE barriers AS 
    SELECT * FROM read_csv_auto('{raw_path}/BarrierstoParticipationinSportPhysicalActivity2022.csv');
""")

con.sql(f"""
    CREATE OR REPLACE TABLE motivations AS 
    SELECT * FROM read_csv_auto('{raw_path}/MotivationstoParticipateinSportPhysicalActivity2022.csv');
""")

Importing Top Sports & Exercise CSVs...
Importing Barriers & Motivations...


In [32]:
# ==========================================
# PART B: SPATIAL LAYERS (GeoJSON & KML)
# ==========================================
print("Importing Geospatial Layers (Facilities, MRT, Parks)...")

# 1. Sports Facilities (Supply)
con.sql(f"""
    CREATE OR REPLACE TABLE facilities AS 
    SELECT * FROM ST_Read('{raw_path}/SportSGSportFacilitiesGEOJSON.geojson');
""")

# 2. MRT Stations (Connectivity)
con.sql(f"""
    CREATE OR REPLACE TABLE mrt_stations AS 
    SELECT * FROM ST_Read('{raw_path}/LTAMRTStationExitGEOJSON.geojson');
""")

# 3. Parks (Nature)
con.sql(f"""
    CREATE OR REPLACE TABLE parks AS 
    SELECT * FROM ST_Read('{raw_path}/NParksParksandNatureReserves.geojson');
""")

# 4. Cycling Infrastructure
# Note: KML support depends on your system's GDAL drivers. 
# If this fails, convert KML to GeoJSON in QGIS first.
try:
    con.sql(f"""
        CREATE OR REPLACE TABLE cycling_paths AS 
        SELECT * FROM ST_Read('{raw_path}/CyclingPathNetworkKML.kml');
    """)
    con.sql(f"""
        CREATE OR REPLACE TABLE hdb_buildings AS 
        SELECT * FROM ST_Read('{raw_path}/HDBExistingBuilding.kml');
    """)
    print("KMLs imported successfully.")
except Exception as e:
    print(f"Skipping KMLs (GDAL error): {e}. Suggest converting KML to GeoJSON externally.")


Importing Geospatial Layers (Facilities, MRT, Parks)...
KMLs imported successfully.
KMLs imported successfully.


In [33]:
# ==========================================
# PART C: TRANSPORT NETWORK (JSON / GeoJSON)
# ==========================================
print("Importing Transport Network...")

# GeoJSON Routes & Stops
con.sql(f"""
    CREATE OR REPLACE TABLE transport_routes AS 
    SELECT * FROM ST_Read('{raw_path}/routes.geojson');
""")

con.sql(f"""
    CREATE OR REPLACE TABLE transport_stops AS 
    SELECT * FROM ST_Read('{raw_path}/stops.geojson');
""")

# Pure JSON files (Services/FirstLast)
con.sql(f"""
    CREATE OR REPLACE TABLE transport_services AS 
    SELECT * FROM read_json_auto('{raw_path}/services.json');
""")


Importing Transport Network...


In [34]:
# ==========================================
# PART D: VERIFICATION
# ==========================================
print("\n--- IMPORT COMPLETE. TABLE SUMMARY ---")
con.sql("SHOW TABLES").show()

# Example Check: See the top 3 rows of the compiled demographics table
print("\n--- PREVIEW: Demographics Data ---")
con.sql("SELECT filename, jogging, \"yoga\" FROM demographics_sports").show()



--- IMPORT COMPLETE. TABLE SUMMARY ---
┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ barriers            │
│ cycling_paths       │
│ demographics_sports │
│ facilities          │
│ hdb_buildings       │
│ motivations         │
│ mrt_stations        │
│ parks               │
│ transport_routes    │
│ transport_services  │
│ transport_stops     │
├─────────────────────┤
│       11 rows       │
└─────────────────────┘


--- PREVIEW: Demographics Data ---
┌──────────────────────────────────────────────────────────────────────────────────────────┬─────────┬───────┐
│                                         filename                                         │ jogging │ yoga  │
│                                         varchar                                          │  int64  │ int64 │
├──────────────────────────────────────────────────────────────────────────────────────────┼─────────┼───────┤
│ /home/amber/Equity_Analysis/data/data_raw/Top

In [35]:
con = duckdb.connect(db_path)
con.sql('SHOW ALL TABLES')

┌──────────────┬─────────┬─────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│   database   │ schema  │        name         │                                                                                                                                                                          column_names                                                                                                                                                